# Neptune + Catalyst
([source](https://github.com/neptune-ai/examples/blob/main/integrations-and-supported-tools/catalyst/notebooks/Neptune_Catalyst.ipynb))

## Install dependencies

In [ ]:
! pip install catalyst==22.02rc0 ipywidgets==7.6.3 neptune-client

## Import libraries

In [ ]:
import os

from collections import OrderedDict
from catalyst import dl
from catalyst.contrib.datasets import MNIST
from torch import nn, optim
from torch.utils.data import DataLoader

## Prepare hyper-parameters

In [ ]:
my_hparams = {"lr": 0.07, "batch_size": 32}

They will be logged automatically.

## Prepare model, criterion, optimizer and data loaders

Define components needed for the Catalyst Runner.

In [ ]:
model = nn.Sequential(nn.Flatten(), nn.Linear(28 * 28, 10))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), my_hparams["lr"])
loaders = OrderedDict(
    {
        "training": DataLoader(
            MNIST(os.getcwd(), train=True, download=True),
            batch_size=my_hparams["batch_size"],
        ),
        "validation": DataLoader(
            MNIST(os.getcwd(), train=False, download=True),
            batch_size=my_hparams["batch_size"],
        ),
    }
)

Notice two keys: `"training"` and `"validation"`. These names are used in Neptune as namespaces to organize your run.

## Create runner

In [ ]:
my_runner = dl.SupervisedRunner()

## Create NeptuneLogger

In [ ]:
from catalyst import dl

neptune_logger = dl.NeptuneLogger(
    api_token="ANONYMOUS",
    project="common/catalyst-integration",
    tags=["notebook-example", "quickstart"],
)

Link above is a link to the run. Click on it, and leave the run tab open - you will get back to it when you start model training.

Few explanations:
1. You need to pass project to the `NeptuneLogger()`, to inform it where to log metadata. Project is a string of this form: `my_workspace/my_project`.
1. There are more parameters to customize logger behavior, check [Catalyst docs](https://catalyst-team.github.io/catalyst/api/loggers.html#neptunelogger) for more details.

----

**Note**

Instead of logging data to the public project `"common/catalyst-integration"` as an anonymous user `"neptuner"` you can log it to your own project.

To do that:
1. Follow the [installation and setup](https://docs.neptune.ai/getting-started/installation) that will show you how to use individual, private api_token.
1. Create new [private project](https://docs.neptune.ai/administration/workspace-project-and-user-management/projects).
1. Pass this project name here, instead of `"common/catalyst-integration"`.

At this point you will be ready to log Catalyst runs to your own project :)

## Pass neptune_logger to the train method of the Runner

In [ ]:
my_runner.train(
    loggers={"neptune": neptune_logger},
    hparams=my_hparams,
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    loaders=loaders,
    num_epochs=10,
    callbacks=[
        dl.AccuracyCallback(input_key="logits", target_key="targets", topk_args=[1]),
        dl.CheckpointCallback(
            logdir="checkpoints",
            loader_key="validation",
            metric_key="loss",
            minimize=True,
        ),
    ],
    valid_loader="validation",
    valid_metric="loss",
    minimize_valid_metric=True,
)

Few explanations:

1. `train` method executes model training, so here you start logging metadata to Neptune.
1. `neptune_logger` is passed to the `loggers` parameter.
1. Notice that `my_hparams` are also passed, to that they can be logged as well.

## Log best model

After training, use `log_artifact()` method to log model checkpoint.

In [ ]:
my_runner.log_artifact(
    path_to_artifact="./checkpoints/model.best.pth", tag="best_model", scope="experiment"
)

## Stop logging

<font color=red>**Warning:**</font><br>
Once you are done logging, you should stop the run using the `stop()` method, which is method of the run object. Run object itself is part of the `neptune_logger`.
This is needed only while logging from a notebook environment. While logging through a script, Neptune automatically stops tracking once the script has completed execution.

In [ ]:
neptune_logger.run.stop()

## Analyze logged metadata in the Neptune UI

Go to the run link and explore metadata (metrics, hparams, model checkpoint) that were logged to the run in Neptune.

Link should look like this: https://app.neptune.ai/common/catalyst-integration/e/CATALYST-1486